In [2]:

import csv
import json
from datetime import datetime

# Data Persistence Files
PRODUCTS_FILE = "products.json"
ORDERS_FILE = "orders.json"
INVOICES_FILE = "invoices.csv"

# Load Data on Program Start
def load_data(filename):
    try:
        with open(filename, 'r') as file:
            return json.load(file)
    except FileNotFoundError:
        return []

def save_data(filename, data):
    with open(filename, 'w') as file:
        json.dump(data, file, indent=4)

products = load_data(PRODUCTS_FILE)
orders = load_data(ORDERS_FILE)

# Admin Functionalities
def add_product():
    product_id = input("Enter Product ID: ")
    name = input("Enter Product Name: ")
    category = input("Enter Product Category: ")
    price = float(input("Enter Product Price: "))
    stock = int(input("Enter Stock Quantity: "))
    
    products.append({"id": product_id, "name": name, "category": category, "price": price, "stock": stock})
    save_data(PRODUCTS_FILE, products)
    print("Product added successfully!")

def update_product():
    product_id = input("Enter Product ID to update: ")
    for product in products:
        if product["id"] == product_id:
            product["name"] = input("Enter New Name: ") or product["name"]
            product["category"] = input("Enter New Category: ") or product["category"]
            product["price"] = float(input("Enter New Price: ") or product["price"])
            product["stock"] = int(input("Enter New Stock Quantity: ") or product["stock"])
            save_data(PRODUCTS_FILE, products)
            print("Product updated successfully!")
            return
    print("Product not found!")

def delete_product():
    product_id = input("Enter Product ID to delete: ")
    global products
    products = [product for product in products if product["id"] != product_id]
    save_data(PRODUCTS_FILE, products)
    print("Product deleted successfully!")

def view_sales_report():
    time_period = input("Enter time period (daily, weekly, monthly): ").lower()
    filtered_orders = []
    current_date = datetime.now()

    for order in orders:
        order_date = datetime.strptime(order["date"], "%Y-%m-%d %H:%M:%S")
        if time_period == "daily" and order_date.date() == current_date.date():
            filtered_orders.append(order)
        elif time_period == "weekly" and (current_date - order_date).days <= 7:
            filtered_orders.append(order)
        elif time_period == "monthly" and (current_date - order_date).days <= 30:
            filtered_orders.append(order)

    for order in filtered_orders:
        print(json.dumps(order, indent=4))

# Customer Functionalities
def browse_catalog():
    print("\nProduct Catalogue:")
    for product in products:
        print(f"ID: {product['id']}, Name: {product['name']}, Price: {product['price']}, Stock: {product['stock']}")

def generate_invoice(order):
    with open(INVOICES_FILE, 'a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["Order ID", "Date", "Item Name", "Quantity", "Price", "Total Price"])
        for item in order["items"]:
            writer.writerow([order["id"], order["date"], item["name"], item["quantity"], item["price"], order["total_price"]])
        writer.writerow([])  # Add a blank line after each order
    print(f"Invoice for Order ID {order['id']} has been generated and saved to {INVOICES_FILE}")

def place_order():
    cart = []
    total_price = 0
    
    while True:
        product_id = input("Enter Product ID to add to cart (or type 'done' to finish): ")
        if product_id.lower() == 'done':
            break

        quantity = int(input("Enter Quantity: "))
        for product in products:
            if product["id"] == product_id:
                if product["stock"] >= quantity:
                    product["stock"] -= quantity
                    cart.append({"id": product_id, "name": product["name"], "quantity": quantity, "price": product["price"]})
                    total_price += product["price"] * quantity
                else:
                    print("Insufficient stock!")
                break

    print("Order Summary:")
    for item in cart:
        print(f"{item['name']} (x{item['quantity']}) - ${item['price'] * item['quantity']}")

    print(f"Total Price: ${total_price}")
    confirm = input("Confirm order? (yes/no): ").lower()
    if confirm == "yes":
        order_id = f"ORD-{len(orders) + 1}"
        order = {
            "id": order_id,
            "date": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
            "items": cart,
            "total_price": total_price
        }
        orders.append(order)
        save_data(ORDERS_FILE, orders)
        save_data(PRODUCTS_FILE, products)
        generate_invoice(order)
        print(f"Order placed successfully! Order ID: {order_id}")

def view_order_history():
    print("\nOrder History:")
    for order in orders:
        print(json.dumps(order, indent=4))

# User Interface
def admin_menu():
    while True:
        print("\nAdmin Menu")
        print("1. Add Product")
        print("2. Update Product")
        print("3. Delete Product")
        print("4. View Sales Report")
        print("5. Logout")
        choice = input("Enter your choice: ")

        if choice == "1":
            add_product()
        elif choice == "2":
            update_product()
        elif choice == "3":
            delete_product()
        elif choice == "4":
            view_sales_report()
        elif choice == "5":
            break
        else:
            print("Invalid choice!")

def customer_menu():
    while True:
        print("\nCustomer Menu")
        print("1. Browse Catalogue")
        print("2. Place Order")
        print("3. View Order History")
        print("4. Logout")
        choice = input("Enter your choice: ")

        if choice == "1":
            browse_catalog()
        elif choice == "2":
            place_order()
        elif choice == "3":
            view_order_history()
        elif choice == "4":
            break
        else:
            print("Invalid choice!")

# Main Program
def main():
    while True:
        print("\nOnline Ordering System")
        print("1. Admin Login")
        print("2. Customer Login")
        print("3. Exit")
        choice = input("Enter your choice: ")

        if choice == "1":
            admin_menu()
        elif choice == "2":
            customer_menu()
        elif choice == "3":
            print("Thank you for using the system!")
            break
        else:
            print("Invalid choice!")

if __name__ == "__main__":
    main()


Online Ordering System
1. Admin Login
2. Customer Login
3. Exit

Admin Menu
1. Add Product
2. Update Product
3. Delete Product
4. View Sales Report
5. Logout
Product added successfully!

Admin Menu
1. Add Product
2. Update Product
3. Delete Product
4. View Sales Report
5. Logout

Online Ordering System
1. Admin Login
2. Customer Login
3. Exit
Thank you for using the system!
